In [1]:
from scipy.special import sph_harm
import numpy as np
import matplotlib.pyplot as plt

from skimage import measure
import meshplot as mp

# Note:
order = m, it varies fast - `m<=np.abs(l)`

degree = l, it varies slow - `l>=1`. 

Atomic orbitals go like 2l+1
https://en.wikipedia.org/wiki/Cubic_harmonic



In [3]:
## start by taking an xyz grid of the unit cube:

spacing = 0.005 #quite fine resolution - slow!
a,b,c = np.meshgrid( 
    np.arange(0,2, spacing)-1,
    np.arange(0,2, spacing)-1,
    np.arange(0,2, spacing)-1,
    indexing='ij'
)

#shape (-1, 3) array of points:
pts = np.vstack([a.ravel(),b.ravel(),c.ravel()]).T

In [11]:
## then (temporarily) convert this into spherical coordinates

def asSpherical(xyz):
    """Convert xyz points to spherical coordinates (r, theta, phi)"""
    #takes list xyz (single coord)
    x       = xyz[:,0]+0.00
    y       = xyz[:,1]+0.00
    z       = xyz[:,2]+0.00
    r       =  np.sqrt(x*x + y*y + z*z)
    theta   =  np.arccos(z/r) #to degrees
    phi     =  np.arctan2(y,x)
    return r, theta, phi

r, theta, phi = asSpherical(pts)

#replace NaNs with 0 
n = 0
theta[np.isnan(theta)]=n
phi[np.isnan(phi)]=n

/var/folders/jh/02165y2n7kq2y5ychxtzcjm40000gn/T/ipykernel_75432/1751958227.py:10: RuntimeWarning: invalid value encountered in true_divide
  theta   =  np.arccos(z/r) #to degrees


In [12]:
## choose one of the spherical harmonics
# remember l>=1, m<=abs(l)

l = 15
m = 13

#calculate the value of the harmonic on the unit sphere at all of 
#the angular coordinates (note 'r' is ignored, hence this is
#evaluated on the unit sphere):
sph_vals = sph_harm(m, l, phi, theta).real

#now ask whether the value of the harmonic function is greater than
#or less than 'r'. This tell us if a point (r,theta,phi) is 'outside' 
#the surface of the harmonic or 'inside' it.
diffs = r - np.abs(sph_vals)
g = diffs.reshape(a.shape)

In [13]:
## finally, we have a grid of values measuring their
# distance to the implicit surface of a spherical harmonic. 
# AKA a signed distance function, so we can apply marching cubes
# to find the isosurface. 

v1, f1, _, _ = measure.marching_cubes(g, 0)

# before plotting, we do want to know the value of the harmonic. 
# I took the absolute values earlier in order to determine the isosurface,
# but now that we have vertices we can just ask whether they are negative
# or positive. 

#convert grid coordinates into universe coordinates:
v1_univ = v1 * spacing - 1
#convert to spherical:
r_v1, theta_v1, phi_v1 = asSpherical(v1_univ)
#evaluate 
cols = sph_harm(m, l, phi_v1, theta_v1).real
cols[np.isnan(cols)]=0

#plot!
mp.offline()
mp.plot(v1, f1, c=cols, filename='./spherical_harmonics.html')


/var/folders/jh/02165y2n7kq2y5ychxtzcjm40000gn/T/ipykernel_75432/1751958227.py:10: RuntimeWarning: invalid value encountered in true_divide
  theta   =  np.arccos(z/r) #to degrees
/Users/ljmartin/miniconda3/envs/compchem/lib/python3.9/site-packages/jupyter_client/session.py:716: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


Plot saved to file ./spherical_harmonics.html.
